In [1]:
import time
import numpy as np
from tqdm import tqdm_notebook as tqdm

# WordEmbeding [CountVectorizer]
В данном пункте сравним скорости работы простого векторного представления предложений. В качестве базового решения выбран ```CountVectorizer``` из пакета ```sklearn```.

На основе ```CountVectorizer``` был написан сообвественый класс ```Vectorizer``` на python. Данный класс повторяет базовые возможности класса ```CountVectorizer```, такие как ```fit``` и ```transform```.

Сревнение производительности проводится для класса ```Vectorizer``` который работает в следующих случаях:
* Простая реализация на pypy3;
* Простая реалзиция на python без компиляции;
* Простая компиляция класса без использования типизации;
* Компиляция класса ```Vectorizer``` с использованием типизации данных.

## Data

In [2]:
import re
import numpy as np

In [3]:
with open('../data/ru.tok', 'r') as f:
    corpus = f.read().lower().splitlines()

## Models

### Python Vectorizer
Данная модель ```Vectorizer``` построенна для повторения базовой функциональности ```CountVectorizer```. В качестве поиска токенов используется такое же регулярное выражение как и в случае дефолтного запуска ```CountVectorizer```.

In [4]:
token_pattern = re.compile(r"(?u)\b\w\w+\b")
class VectorizerPypy:
    def __init__(self):
        self.WordToInt = dict()
        pass
    
    def split_sent(self, sent):
        return token_pattern.findall(sent)
    
    def fit(self, corpus):
        for sent in corpus:
            words = self.split_sent(sent)
            for word in words:
                if word not in self.WordToInt:
                    self.WordToInt[word] = len(self.WordToInt)
        return

    def get_feature_names(self):
        return list(self.WordToInt.keys())
    
    def transform(self, corpus):
        transformed = np.zeros([len(corpus), len(self.WordToInt)], dtype=np.int64)
        for i, sent in enumerate(corpus):
            words = self.split_sent(sent)
            row = transformed[i]
            LocalVocab = dict()
            for word in words:
                if word in self.WordToInt:
                    ind = self.WordToInt[word]
                    if ind not in LocalVocab:
                        LocalVocab[ind] = 1
                    else:
                        LocalVocab[ind] += 1
                    
            for ind in LocalVocab:
                row[ind]=LocalVocab[ind];
                    
        return transformed
    
    def fit_transform(self, corpus):
        self._fit(corpus)
        return self._transform(corpus)

## Comparison
В эксперименте сравнивается время метода ```fit```, а также метода ```transform``` для всех моделей. Для оценки времени производится усреднение по нескольким независимым вызовам данных методов. Каждый раз вызов метода ```fit``` производится на новом объекте рассматриваемого класса. Метод ```transform``` вызывается также каждый раз на новом объекте рассматриваемого класса после вызова метода ```fit```.

In [5]:
number_of_experiment = 200

#### Python Vectorizer
Реализация дефолтного ```CountVectorizer``` на python без компиляциии кода.

In [6]:
list_of_fit_times = []
list_of_transform_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    model = VectorizerPypy()
    
    start = time.time()
    model.fit(corpus)
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    start = time.time()
    _ = model.transform(corpus)
    end = time.time()
    
    list_of_transform_times.append(end-start)
    
    iterable.set_postfix_str('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))

print('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))

/usr/local/Cellar/pypy3/7.3.0/libexec/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



(fit=0.37626678109169004, transform=3.1669243133068083)


## Result

Оценка времени выполнялась для выборки, которая состояла из $79582$ строк ($16.8$mb текста). Всего в данном тексте содержится $76777$ различных токенов, которые были найдены всемы моделями и добавлены в словарь.


Для чистоты эксперимента время предаставленное в таблице является устредненным по $200$ вызовам функции ```fit``` и $200$ вызовам функции ```transform```.

| Функция  | Время fit | Время transform |
| ------------- | ------------- | ------------- |
| VectorizerPypy  | 376 ms | 3167 ms |

